# Data Analysis - MoviesLes dataset

In [113]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

movies = pd.read_csv('data/movies.csv')
ratings = pd.read_csv('data/ratings.csv')

## 데이터 파악

In [72]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [75]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [18]:
movies.columns

Index(['movieId', 'title', 'genres'], dtype='object')

In [17]:
ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [131]:
movies.shape

(9742, 3)

In [10]:
ratings.shape

(100836, 4)

### 중복값 제거

In [132]:
print(ratings.drop_duplicates('userId', keep='first')) # 610
print(movies.drop_duplicates('movieId', keep='first')) # 9742

       userId  movieId  rating   timestamp
0           1        1     4.0   964982703
232         2      318     3.0  1445714835
261         3       31     0.5  1306463578
300         4       21     3.0   986935199
516         5        1     4.0   847434962
...       ...      ...     ...         ...
97364     606        1     2.5  1349082950
98479     607        1     4.0   964744033
98666     608        1     2.5  1117408267
99497     609        1     3.0   847221025
99534     610        1     5.0  1479542900

[610 rows x 4 columns]
      movieId                                      title  \
0           1                           Toy Story (1995)   
1           2                             Jumanji (1995)   
2           3                    Grumpier Old Men (1995)   
3           4                   Waiting to Exhale (1995)   
4           5         Father of the Bride Part II (1995)   
...       ...                                        ...   
9737   193581  Black Butler: Book of the

### 같은 컬럼을 기준으로 데이터 병합

In [83]:
data = pd.merge(movies, ratings)
display(data)
data.shape

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483
...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184,4.0,1537109082
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184,3.5,1537109545
100833,193585,Flint (2017),Drama,184,3.5,1537109805
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5,1537110021


(100836, 6)

## [문제 1] 사용자가 평가한 모든 영화의 전체 평균 평점

In [285]:
movies_mean = ratings['rating'].mean()
print(movies_mean)

3.501556983616962


## [문제 2] 각 사용자별 평균 평점

In [334]:
user_rating_mean = ratings['rating'].groupby(ratings['userId']).mean()
display(user_rating_mean)

userId
1      4.366379
2      3.948276
3      2.435897
4      3.555556
5      3.636364
         ...   
606    3.657399
607    3.786096
608    3.134176
609    3.270270
610    3.688556
Name: rating, Length: 610, dtype: float64

## [문제 3] 각 영화별 평균 평점

### ❌ ratings이라는 DataFrame에서만 구했을 경우, Length: 9724
* ratings: 평점이 있는 영화만 존재
* movies: 모든 영화가 존재
* data: 공통 컬럼인 `movieId`를 기준으로 ratings, movies를 합침, 결국 평점이 없는 영화는 자동 배제됨
* 따라서 movies의 데이터는 총 9742개인 반면, 아래의 코드를 실행 시 데이터의 갯수는 9724개로 줄어들었음

In [338]:
movie_rating_mean = ratings['rating'].groupby(ratings['movieId']).mean()
movie_rating_mean # Length: 9724

movieId
1         3.920930
2         3.431818
3         3.259615
4         2.357143
5         3.071429
            ...   
193581    4.000000
193583    3.500000
193585    3.500000
193587    3.500000
193609    4.000000
Name: rating, Length: 9724, dtype: float64

### ⭕️ 평점이 매겨지지 않은 영화도 포함하여 다시 계산할 경우, Length: 9742
* moives를 기준으로 movies와 movie_rating_mean을 다시 병합
* `left_on='movieId`': movies의 `moiveId`라는 컬럼을 기준으로 병합
* `right_index=True`: movie_rating_mean의 인덱스(`movieId`)를 기준으로 병합
* `how='left'`: 왼쪽에 존재한 movies에, 오른쪽에 위치한 movie_rating_mean을 갖다붙임

In [339]:
df = pd.merge(movies, movie_rating_mean, left_on='movieId', right_index=True, how='left')
movie_rating_mean = df[['movieId', 'title', 'rating']]
display(movie_rating_mean) # Length: 9742

# 평점이 매겨지지 않은 영화 확인 - NaN
movie_rating_mean.loc[movie_rating_mean['rating'].isna(), :] # 18개 = 9742 - 9724


,movieId,title,rating
0,1,Toy Story (1995),3.920930
1,2,Jumanji (1995),3.431818
2,3,Grumpier Old Men (1995),3.259615
3,4,Waiting to Exhale (1995),2.357143
4,5,Father of the Bride Part II (1995),3.071429
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),4.000000
9738,193583,No Game No Life: Zero (2017),3.500000
9739,193585,Flint (2017),3.500000
9740,193587,Bungo Stray Dogs: Dead Apple (2018),3.500000


,movieId,title,rating
816,1076,"Innocents, The (1961)",NaN
2211,2939,Niagara (1953),NaN
2499,3338,For All Mankind (1989),NaN
2587,3456,"Color of Paradise, The (Rang-e khoda) (1999)",NaN
3118,4194,I Know Where I'm Going! (1945),NaN
4037,5721,"Chosen, The (1981)",NaN
4506,6668,"Road Home, The (Wo de fu qin mu qin) (1999)",NaN
4598,6849,Scrooge (1970),NaN
4704,7020,Proof (1991),NaN
5020,7792,"Parallax View, The (1974)",NaN


## [문제 4] 평균 평점이 가장 높은 영화의 제목
**(단, 동률이 있을 경우 모두 출력하되, title을 기준으로 오름차순 정렬)**

In [258]:
movie_max = movie_rating_mean.loc[movie_rating_mean['rating'].max() == movie_rating_mean['rating'], :]
result = movie_max.sort_values(by='title', ascending=True)
display(result)

,movieId,title,rating
5690,27751,'Salem's Lot (2004),5.0
7332,77846,12 Angry Men (1997),5.0
9046,141816,12 Chairs (1976),5.0
3893,5468,20 Million Miles to Earth (1957),5.0
5639,27373,61* (2001),5.0
...,...,...,...
9711,187717,Won't You Be My Neighbor? (2018),5.0
8355,108795,Wonder Woman (2009),5.0
9289,158398,World of Glory (1991),5.0
9560,173351,Wow! A Talking Fish! (1983),5.0


## [문제 5] Comedy 영화 중 가장 평점이 낮은 영화의 제목
**(단, 동률이 있을 경우 모두 출력하되, title을 기준으로 오름차순 정렬)**

In [342]:
movie_rating_mean = df[['movieId', 'title', 'rating', 'genres']]

comedy_movie = movie_rating_mean.loc[movie_rating_mean['genres'].str.contains('Comedy'), :]
comedy_min = comedy_movie.loc[comedy_movie['rating'].min() == comedy_movie['rating'], :]
result = comedy_min.sort_values(by='title', ascending=True)
display(result)

,movieId,title,rating,genres
8893,134528,Aloha (2015),0.5,Comedy|Drama|Romance
5777,31422,Are We There Yet? (2005),0.5,Children|Comedy
7762,91414,Arthur Christmas (2011),0.5,Animation|Children|Comedy|Drama
9419,165645,Bad Santa 2 (2016),0.5,Comedy
4439,6557,Born to Be Wild (1995),0.5,Adventure|Children|Comedy|Drama
5409,25782,Boudu Saved From Drowning (Boudu sauvé des eau...,0.5,Comedy
6554,54934,"Brothers Solomon, The (2007)",0.5,Comedy
5453,26095,"Carabineers, The (Carabiniers, Les) (1963)",0.5,Comedy|Drama|War
6545,54768,Daddy Day Camp (2007),0.5,Children|Comedy
4881,7312,"Follow Me, Boys! (1966)",0.5,Comedy|Drama


## [문제 6] 2015년도에 평가된 모든 Romance 영화의 평균 평점


### Romance 영화의 평균 평점

In [343]:
romance_movie = movie_rating_mean.loc[movie_rating_mean['genres'].str.contains('Romance'), :]
display(romance_movie)
romance_movie_tm = pd.merge(romance_movie, ratings['timestamp'], left_on='movieId', right_index=True, how='left')
display(romance_movie_tm)

,movieId,title,rating,genres
2,3,Grumpier Old Men (1995),3.259615,Comedy|Romance
3,4,Waiting to Exhale (1995),2.357143,Comedy|Drama|Romance
6,7,Sabrina (1995),3.185185,Comedy|Romance
10,11,"American President, The (1995)",3.671429,Comedy|Drama|Romance
14,15,Cutthroat Island (1995),3.000000,Action|Adventure|Romance
...,...,...,...,...
9639,179511,Emerald Green (2016),4.000000,Adventure|Drama|Fantasy|Romance
9660,181315,Phantom Thread (2017),3.500000,Drama|Romance
9691,184349,Elsa & Fred (2005),3.500000,Comedy|Drama|Romance
9715,188751,Mamma Mia: Here We Go Again! (2018),4.500000,Comedy|Romance


,movieId,title,rating,genres,timestamp
2,3,Grumpier Old Men (1995),3.259615,Comedy|Romance,964983815.0
3,4,Waiting to Exhale (1995),2.357143,Comedy|Drama|Romance,964982931.0
6,7,Sabrina (1995),3.185185,Comedy|Romance,964982176.0
10,11,"American President, The (1995)",3.671429,Comedy|Drama|Romance,964981208.0
14,15,Cutthroat Island (1995),3.000000,Action|Adventure|Romance,964981680.0
...,...,...,...,...,...
9639,179511,Emerald Green (2016),4.000000,Adventure|Drama|Fantasy|Romance,NaN
9660,181315,Phantom Thread (2017),3.500000,Drama|Romance,NaN
9691,184349,Elsa & Fred (2005),3.500000,Comedy|Drama|Romance,NaN
9715,188751,Mamma Mia: Here We Go Again! (2018),4.500000,Comedy|Romance,NaN


### timestamp 다루기

In [344]:
import time
start = time.mktime(time.strptime("2015-01-01 00:00:00","%Y-%m-%d %H:%M:%S"))
end = time.mktime(time.strptime("2016-01-01 00:00:00","%Y-%m-%d %H:%M:%S"))

result = romance_movie_tm.loc[(romance_movie_tm["timestamp"] >= start) & (romance_movie_tm["timestamp"] < end),"rating"]
display(result)

198     3.892857
202     3.477778
203     3.340909
205     3.000000
213     3.722222
216     3.453488
222     2.600000
2584    3.000000
2637    3.400000
2690    3.750000
2708    3.264706
5402    3.000000
5405    4.000000
5406    4.500000
5411    3.500000
6005    3.534091
6012    2.000000
6140    3.000000
6165    4.117647
6186    4.250000
6204    3.250000
6210    2.590909
6282    3.500000
6285    2.500000
6362    3.000000
6574    3.000000
6652    3.000000
6809    3.000000
6968    2.818182
7087    3.666667
7094    3.500000
7351    2.833333
7416    3.796296
7417    3.500000
7420    4.000000
7421    3.833333
7632    2.750000
Name: rating, dtype: float64

### 2015년도에 평가된 모든 Romance 영화의 평균 평점

In [328]:
answer = result.mean()
print(answer)

3.3335788917526257
